In [14]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sb
import multiprocessing as mp
import os
import cupy as cp
import cuml as cm

In [15]:
data = pd.read_csv('dataset.csv', index_col=0)

In [16]:
data.reset_index(inplace=True, drop=True)
data.head()

,geneID,x,y,MIDCount,ExonCount,CellID
0,Tmlhe,10618,12270,1,1,6954
1,Tmlhe,8222,15645,1,1,47144
2,Tmlhe,8917,15028,1,1,39597
3,Tmlhe,9046,14970,1,1,38748
4,Tmlhe,10599,14981,1,1,39019


In [17]:
data['CellID'] = data['CellID'].astype(np.uint16)
data['ExonCount'] = data['ExonCount'].astype(np.uint8)
data['MIDCount'] = data['MIDCount'].astype(np.uint8)
data['x'] = data['x'].astype(np.uint16)
data['y'] = data['y'].astype(np.uint16)

In [18]:
unique_cells = data['CellID'].unique()
unique_genes = data['geneID'].unique()

In [34]:
def midcount_filling() -> int:
    gpu_matrix = cp.zeros((len(unique_cells), len(unique_genes)), dtype=cp.uint8)
    cell_indices = pd.factorize(data['CellID'])[0] 
    gene_indices = pd.factorize(data['geneID'])[0]

    gpu_cell_indices = cp.array(cell_indices, dtype=cp.uint16)
    gpu_gene_indices = cp.array(gene_indices, dtype=cp.uint16)
    gpu_midcount = cp.array(data['MIDCount'].values, dtype=cp.uint8)

    gpu_matrix[gpu_cell_indices, gpu_gene_indices] = gpu_midcount

    tmp_matrix = cp.asnumpy(gpu_matrix)

    unloged_matrix = pd.DataFrame(tmp_matrix, index=unique_cells, columns=unique_genes, dtype=np.uint8)
    unloged_matrix.to_csv('unloged_matrix.csv', index=False)

    unloged_matrix.info()
    return unloged_matrix.last_valid_index()

unloged_matrix_number_of_rows = midcount_filling()

<class 'pandas.core.frame.DataFrame'>
Index: 62725 entries, 6954 to 22645
Columns: 20753 entries, Tmlhe to Gm42418
dtypes: uint8(20753)
memory usage: 1.2 GB


In [40]:
unloged_matrix_number_of_rows

22645

In [41]:
half_rows = unloged_matrix_number_of_rows // 2
second_half_rows = unloged_matrix_number_of_rows - half_rows

def loging_matrix(data_chunk, mode='w') -> None:
    gpu_matrix = cp.array(data_chunk.values, dtype=cp.float32)

    gpu_matrix[gpu_matrix == 0] = -1
    gpu_matrix = cp.where(gpu_matrix == -1, -1, cp.log(gpu_matrix))

    tmp_matrix = cp.asnumpy(gpu_matrix)

    loged_matrix = pd.DataFrame(tmp_matrix, index=data_chunk.index, columns=data_chunk.columns, dtype=np.float32)
    loged_matrix.to_csv('loged_matrix.csv', mode=mode, header=(mode == 'w'), index=False)

def first_part() -> None:
    print(f'loading first part')
    first_half = pd.read_csv('unloged_matrix.csv', nrows=half_rows)
    print(f'first part is loaded')
    print(f'starting loging first part')
    loging_matrix(first_half, mode='w')
    print(f'first part is loged')

def second_part() -> None:
    print(f'loading second part')
    second_half = pd.read_csv('unloged_matrix.csv', skiprows=half_rows, nrows=second_half_rows)
    print(f'second part is loaded')
    print(f'starting loging second part')
    loging_matrix(second_half, mode='a')
    print(f'second part is loged') 

first_part()
second_part()

loading first part
first part is loaded
starting loging first part
first part is loged
loading second part
second part is loaded
starting loging second part
second part is loged


In [45]:
from cuml.cluster import HDBSCAN

In [46]:
help(HDBSCAN)

Help on class HDBSCAN in module cuml.cluster.hdbscan.hdbscan:

class HDBSCAN(cuml.internals.base.UniversalBase, cuml.internals.mixins.ClusterMixin, cuml.internals.mixins.CMajorInputTagMixin)
 |  HDBSCAN(*, min_cluster_size=5, min_samples=None, cluster_selection_epsilon=0.0, max_cluster_size=0, metric='euclidean', alpha=1.0, p=2, cluster_selection_method='eom', allow_single_cluster=False, gen_min_span_tree=False, handle=None, verbose=False, connectivity='knn', output_type=None, prediction_data=False)
 |  
 |  HDBSCAN Clustering
 |  
 |  Recursively merges the pair of clusters that minimally increases a
 |  given linkage distance.
 |  
 |  Note that while the algorithm is generally deterministic and should
 |  provide matching results between RAPIDS and the Scikit-learn Contrib
 |  versions, the construction of the k-nearest neighbors graph and
 |  minimum spanning tree can introduce differences between the two
 |  algorithms, especially when several nearest neighbors around a
 |  point 

In [48]:
def clustering() -> None:
    print(f'Starting to load matrix')
    loged_matrix = pd.read_csv('loged_matrix.csv', index_col=0)
    print(f'Finished loading matrix')
    print(f'Converting matrix to cp.array')
    data = cp.array(loged_matrix.values)
    print(f'Finished conversion')

    hdbscan = HDBSCAN(min_cluster_size=100)
    print(f'Fitting data')
    hdbscan.fit(data)
    print(f'Finished fitting data')

clustering()

Starting to load matrix
Finished loading matrix
Converting matrix to cp.array
Finished conversion
Fitting data


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/petar/anaconda3/envs/rapids-24.08/include/rmm/mr/device/cuda_memory_resource.hpp:60: cudaErrorMemoryAllocation out of memory